In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## PolyA gene detection

What fraction of polyA genes from the annotation do we detect with >= 1 TPM in any library?

In [2]:
df = pd.read_csv('human_talon_abundance.tsv', sep='\t')
df = get_tpm_table(df, how='gene', min_tpm=1)

39285
Total # genes detected: 39285
# genes >= 1 tpm: 33946


In [3]:
gene_df = get_gtf_info(how='gene')

In [6]:
polya_biotypes = ['protein_coding', 'pseudogene', 'lncRNA']
polya_genes = gene_df.loc[gene_df.biotype_category.isin(polya_biotypes), 'gid'].tolist()
n_polya = len(polya_genes)
temp = df.loc[df.index.isin(polya_genes)]
n_det_polya = len(temp.index)

print('Detected {} / {} annotated polyA genes'.format(n_det_polya, n_polya))

Detected 32160 / 49472 annotated polyA genes


How many genes of each biotype are there in the annotation?

In [8]:
temp = gene_df[['gid', 'biotype_category']].groupby('biotype_category').count()
temp.reset_index(inplace=True)
temp.rename({'gid': 'counts'}, axis=1, inplace=True)
temp

,biotype_category,counts
0,lncRNA,15006
1,miRNA,1881
2,other,7426
3,protein_coding,19969
4,pseudogene,14497


How many genes of each biotype do we detect?

In [ ]:
df = df.merge(gene_df, how='left', left_index=True, right_on='gid')
temp = df[['gid', 'biotype_category']].groupby('biotype_category').count()
temp.reset_index(inplace=True)
temp.rename({'gid': 'counts'}, axis=1, inplace=True)
temp